# core

> This is where users start to interact with the underlying LLM and agents.

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# install dependencies
# %pip install replicate
# %pip install gradio
# %pip install langchain

In [4]:
#| export
# import langchain

import os
import llm_QA


/Volumes/SSD2TB/Google Drive/Work/MachineLearning/SciAI/nbs/llm_QA.py:197: SyntaxWarning: invalid escape sequence '\<'
  },


In [5]:
#| export

model = llm_QA.LLM("meta-llama", "TogetherAI")

In [ ]:
#| export
# A chatbot that allows users to interact with an LLM
import gradio as gr

def taskRouter(input, history):
    prompt = input["text"]
    files = input["files"]
    messages = [history,
        {
            "role": "user",
            "content": prompt
        }
    ]

    response = model.get_llm_response(messages)
    #history = [messages, 
    #    {
    #        "role": "assistant",
    #        "content": response
    #    }
    #]
    return response

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value["value"])
    else:
        print("You downvoted this response: " + data.value["value"])

with gr.Blocks() as sciChat:
    chatbot = gr.Chatbot(
        height=300,
        type="messages",
        placeholder="<strong>Your Personal Science AI Assistant</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(
        fn=taskRouter,
        type="messages",
        chatbot=chatbot,
        textbox=gr.MultimodalTextbox(placeholder="How can I help?", container=False, scale=7),
        title="SciAI Assistant",
        description="Ask me any question or carry out a task",
        theme="soft",
        examples=[{"text": "Hello", "files": []},{"text": "What do you see in this image?", "files": []}],
        cache_examples=True,
        multimodal=True
    )
    
sciChat.launch(share=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()